<a href="https://colab.research.google.com/github/thanit456/BayesianNetwork/blob/master/propablistic_model_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install --upgrade tensorflow-probability

Requirement already up-to-date: tensorflow-probability in /usr/local/lib/python3.6/dist-packages (0.10.0rc0)


In [0]:
import numpy as np
import tensorflow_probability as tfp
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [0]:
n = int(64)
scale_tril = np.array([[1.6180, 0.],
                       [-2.7183, 3.1415]]).astype(np.float32)
scale_noise = 0.01
x = tfp.distributions.Normal(loc=0, scale=1).sample([n, 2])                    
eps = tfp.distributions.Normal(loc=0, scale=scale_noise).sample([n, 2])
y = tfp.distributions.Bernoulli(logits=tf.reshape(
    tf.matmul(x, scale_tril) + eps,
    shape=[n, 1, 2, 1]
)).sample()

In [0]:
event_shape = y.shape[1:].as_list()
model = Sequential([
                    Dense(tfp.layers.IndependentBernoulli.params_size(event_shape)),
                    tfp.layers.IndependentBernoulli(event_shape),
])

In [0]:
model.compile(optimizer=Adam(learning_rate=0.5),
              loss=lambda y, model: -model.log_prob(y),
              metrics=[])

In [24]:
batch_size=100
model.fit(x, y, 
          batch_size=batch_size,
          epochs=10,
          steps_per_epoch=n,
          shuffle=True)
print(model.get_weights())

Epoch 1/10
10/64 [===>..........................] - 0s 4ms/step - loss: 0.8574
[array([[ 1.5151271,  1.6764711],
       [-3.4653256,  4.0706406]], dtype=float32), array([ 1.0106779, -0.1510009], dtype=float32)]
